# Packages

In [ ]:
import sspa
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import gseapy

# Data

In [ ]:
# processed proteomics data
prot = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Processing/Processing_Cleaned/cleaned_metabolomics_data_covid.csv')

# processed metaboloimcs data
metab = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Processing/Processing_Cleaned/cleaned_proteomics_data_covid.csv')

# Standard Metabolomics Identifer Harmonisation with Metaboanalyst for ssPA and PathIntegrate

### Creating mapping table

In [ ]:
compound_names = metab.columns.tolist()

In [ ]:
compound_names = processed_data.columns.tolist()
conversion_table = sspa.identifier_conversion(input_type="name", compound_list=compound_names)

In [ ]:
conversion_table['ChEBI'] = pd.to_numeric(conversion_table['ChEBI'], errors='coerce')

conversion_table.dropna(subset=['ChEBI'], inplace=True)
conversion_table['ChEBI'] = conversion_table['ChEBI'].astype('Int64')


### Using mapping table to convert

In [ ]:
processed_data_mapped = sspa.map_identifiers(conversion_table, output_id_type="ChEBI", matrix=processed_data)

# Importing Pathway Database

In [ ]:
# We will import the metabolite pathways from the Reactome database
# We must specify one of the Reactome organism names
# This returns a GMT format pandas DataFrame containing the pathway information
reactome_pathways  = sspa.process_reactome(organism="Homo sapiens")

### Checking IDs mapping to pathway databsase

In [ ]:
## if using Reactome database as the pathway database
# count all compounds in the dataset
print(len(compound_names), "compounds in the dataset")

# find how many input compound names in the dataset had a matching ChEBI ID
chebi_matches = conversion_table[(conversion_table["Comment"] == "1") & (conversion_table["ChEBI"].isnull()==False)]["ChEBI"]
print(len(chebi_matches), "compounds from the dataset that have ChEBI IDs")

# count all unique compounds in the Reactome database
all_reactome_cpds = set(sum(sspa.utils.pathwaydf_to_dict(reactome_pathways).values(), []))
print(len(all_reactome_cpds), "total unique compounds in Reactome")

# find the intesect between all reactome compounds and all ChEBI IDs annotated to the dataset
mapped_annotated_cpds = set(processed_data_mapped.columns) & all_reactome_cpds
print(len(mapped_annotated_cpds), "compounds present in both the dataset and Reactome pathways")

### Barchart

In [ ]:
sns.set_context('notebook')
sns.set_style('ticks')
sns.barplot(y=[len(compound_names), len(chebi_matches), len(mapped_annotated_cpds)], x=['Original', 'Mapping to CHEBI', 'Annotated to Reactome'])
plt.tight_layout()
plt.show()

### Interactive Heatmap

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook"

df = pd.DataFrame(compound_names, columns=['Original_ID'])
df["Matched_ID"] = df['Original_ID'].map(dict(zip(conversion_table["Query"], conversion_table["ChEBI"])))
df["In_pathway"] = [i if i in mapped_annotated_cpds else "NA" for i in df["Matched_ID"] ]
df = df.replace({"NA":0})
df[df != 0] = 1
df = df.astype("float")
df.index = compound_names

fig = px.bar(df)
fig.show(renderer="colab")

### Saving the MetaboAnalyst Mapped Dataset

In [ ]:
metab.to_csv('/Users/judepops/Documents/PathIntegrate/Code/Pathway_Analysis/COVID_Met_ChEBI_Final.csv')

# A) Performing single-sample pathway analysis (ssPA)

### Using the SVD scores method that will be used throughout this project

In [ ]:
kpca_scores = sspa.sspa_SVD(reactome_pathways, min_entity=3, random_state=1).fit_transform(processed_data_mapped)

# Inspect the pathway score matrix
kpca_scores

In [ ]:
# Pathway-based PCA plot

# Normalise kPCA scores
kpca_scores_norm = pd.DataFrame(StandardScaler().fit_transform(kpca_scores))

# Perform two component PCA using sklearn
pca = PCA(n_components=2)
pca_res = pca.fit_transform(kpca_scores_norm)

# determine the variance explained by the first 2 components
pca.explained_variance_ratio_

# Plot the first two components as a scatterplot
plt.style.use("default")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
sns.scatterplot(x=pca_res[:, 0 ], y=pca_res[:, 1], hue=covid_data["Group"], ax=ax1, s=50, alpha=0.5)
sns.scatterplot(x=pca_res[:, 0 ], y=pca_res[:, 1], hue=covid_data["WHO_status"], ax=ax2, s=50, alpha=0.5)

# Set axis labels
ax1.set_xlabel('PC1 (' + str(round(pca.explained_variance_ratio_[0]*100,2)) + '%)')
ax1.set_ylabel('PC2 (' + str(round(pca.explained_variance_ratio_[1]*100,2)) + '%)')
ax2.set_xlabel('PC1 (' + str(round(pca.explained_variance_ratio_[0]*100,2)) + '%)')
ax2.set_ylabel('PC2 (' + str(round(pca.explained_variance_ratio_[1]*100,2)) + '%)')

plt.tight_layout()
# plt.savefig(".kpca_pca_plots.png", dpi=350, bbox_inches="tight")

plt.show()

In [ ]:
# get the loadings of the pathway-based PCA
loadings = pd.DataFrame(pca.components_.T* np.sqrt(pca.explained_variance_)*10,columns=['PC1','PC2'], index=kpca_scores.columns)

# add pathway names to the loadings dataframe
loadings['Pathway'] = loadings.index.map(dict(zip(reactome_pathways.index, reactome_pathways['Pathway_name'])))

# subset top 10 loadings for visual clarity
loadings_top_10 = loadings.sort_values(by='PC1').iloc[0:10, :]

# Plot the first two components as a scatterplot
fig = px.scatter(x=pca_res[:, 0 ], y=pca_res[:, 1], color=covid_data["Group"], labels={'x':'PC1', 'y':'PC2'})

# Plot lines to origin representing the loadings
for i in range(0, loadings_top_10.shape[0]):
  fig.add_trace(go.Scatter(x=[0, loadings_top_10.iloc[i, :]['PC1']], y=[0, loadings_top_10.iloc[i, :]['PC2']],
                           line_color='black', marker_size=0, text=loadings_top_10.iloc[i, :]['Pathway']))

fig.update_layout(width=600, height=600, yaxis_range=[-10, 10], xaxis_range=[-15, 15], showlegend=False)

fig.show()


In [ ]:
# Plot a heatmap using the pathway scores
g = sns.clustermap(kpca_scores_norm.T,
               cmap="RdBu_r",
               z_score=1,
              col_colors = ["tab:red" if i == "COVID19 " else "tab:green" for i in covid_data["Group"]],
              xticklabels=False,
              yticklabels=False)
g.ax_heatmap.set_xlabel("Samples")
g.ax_heatmap.set_ylabel("Pathways")

# plt.savefig("kpca_heatmap.png", dpi=350, bbox_inches="tight")
plt.show()

# B) Using PathIntegrate for Machine Learning on the ssPA scores matrix calculated by ssPA

Additional Dependencies

In [ ]:
import pathintegrate

### Loading the identifier-mapped datasets back in (before ssPA has been run)

In [ ]:
metab = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Pathway_Analysis/COVID_Met_ChEBI_Final.csv')
prot = pd.read_csv('/Users/judepops/Documents/PathIntegrate/Code/Pathway_Analysis/COVID_Pro_UniProt_Final.csv')

In [ ]:
# Dropping irrelevant metadata columns
prot = prot.drop(columns=['Who', 'Race', 'Age', 'Group', 'Age_Group', 'Race_Group'])
metab = metab.drop(columns=['Who', 'Race', 'Age', 'Group', 'Age_Group', 'Race_Group'])

### Make sure only matching samples are used

In [ ]:
prot.set_index('sample_id', inplace=True)
metab.set_index('sample_id', inplace=True)
metab['Condition_Group'].value_counts()

In [ ]:
common_indices = prot.index.intersection(metab.index)
prot = prot.loc[common_indices]
metab = metab.loc[common_indices]

### Loading Reactome Pathways

In [ ]:

mo_paths = sspa.process_reactome(
    organism='Homo sapiens',
    download_latest=True,
    omics_type='multiomics',
    filepath='.' # save to current directory
)

# Inititating PathIntegrate model

### Using SVD and min 4 compound per pathway

In [ ]:
pi_model = pathintegrate.PathIntegrate(
    omics_data={'Metabolomics': metab.iloc[:, :-1], 'Proteomics':prot.iloc[:, :-1]}, # dictionary of multi-omics DataFrames and names for each omics
    metadata=prot['Condition_Group'], # metadata column
    pathway_source=mo_paths, # pathways dataframe
    sspa_scoring=sspa.sspa_SVD, # ssPA method, see ssPA package for options
    min_coverage=4) # minimum number of molecules mapping per pathway to be included

# Fitting a cross-validated single-vew PathIntegrate model for machine learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection a train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import f1_score, roc_auc_score

### Making sure the labels are binary and unique

In [ ]:
labels, y = np.unique(prot['Condition_Group'], return_inverse=True)

### Train test split for each multi-omics dataset

In [ ]:
X_train_prot, X_test_prot, y_train, y_test = train_test_split(prot, y, test_size=0.33, random_state=0, stratify=y)
# use the indices from the protein data to subset the metabolite data
X_train_met, X_test_met = metab.loc[X_train_prot.index, :], metab.loc[X_test_prot.index, :]

In [ ]:
# instantiate a model with the training data only
pi_model = pathintegrate.PathIntegrate(
    omics_data={'Metabolomics_train': X_train_met, 'Proteomics_train': X_train_prot.iloc[:, :-1]},
    metadata=y_train,
    pathway_source=mo_paths,
    sspa_scoring=sspa.sspa_SVD,
    min_coverage=4)

### Getting cross-validated performance metrics from training data

In [ ]:
from sklearn.linear_model import LogisticRegression
cv_single_view = pi_model.SingleViewCV(
    LogisticRegression,
    model_params={'random_state':0, 'max_iter':500},
    cv_params={'cv':5, 'scoring':'f1', 'verbose':2})

print('Mean cross-validated F1 score: ', np.mean(cv_single_view))

### Performing grid search cross valridation to get hyperparameters

In [ ]:

param_grid = {
    "model__C": np.logspace(-4, 4, 4), # every parameter must begin with "model__"
}

sv_grid_search = pi_model.SingleViewGridSearchCV(
    model=LogisticRegression,
    param_grid=param_grid,
    grid_search_params={'cv':3, 'scoring':'roc_auc', 'verbose':2}
    )

In [ ]:
best_params = sv_grid_search.best_params_


In [ ]:
sv_grid_search.best_score_


In [ ]:
sv_grid_search.best_estimator_

### fittign optimised model with best parameters

In [ ]:
sv_grid_search.best_params_
sv_tuned = pi_model.SingleView(
    model=LogisticRegression,
    model_params={'C': best_params['model__C'], 'random_state': 0, 'max_iter': 500}
)

### visualising sspa scores

In [ ]:
sv_tuned.sspa_scores

### predicting on unseen test set

In [ ]:
# generate multi-omics pathway scores for test set
concat_data = pd.concat({'Metabolomics_test': X_test_met, 'Proteomics_test': X_test_prot.iloc[:, :-1]}.values(), axis=1)

pipe_sv = Pipeline([
            ('Scaler', StandardScaler().set_output(transform="pandas")),
            ('sspa', pi_model.sspa_method(pi_model.pathway_source, pi_model.min_coverage)),
        ])

test_set_scores = pipe_sv.fit_transform(concat_data)

# predict using the test set scores
sv_pred = sv_tuned.predict(test_set_scores)

# evalaute the prediction
test_set_f1 = f1_score(y_test, sv_pred)
print(test_set_f1)

### confusion matrix

In [ ]:

# display confusion matrix for test set
from sklearn.metrics import confusion_matrix
sns.heatmap(
    data=confusion_matrix(y_test, sv_pred),
    annot=True,
    square=True,
    cmap='Blues',
    )

# set x and y labels
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


### roc curve

In [ ]:
# plot ROC curve for test set
from sklearn.metrics import roc_curve, auc

sns.set_style('darkgrid')
fpr, tpr, _ = roc_curve(y_test, sv_tuned.predict_proba(test_set_scores)[:, 1])
plt.plot(fpr, tpr, label='Test set')

# plot ROC curve for training set
fpr, tpr, _ = roc_curve(y_train, sv_tuned.predict_proba(sv_tuned.sspa_scores)[:, 1])
plt.plot(fpr, tpr, label='Training set')

# add roc score to plot
plt.plot([0,1], [0, 1], linestyle='--', label='Random', c='k')
plt.title('ROC curve')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')

# add legend
plt.legend()
plt.show()
